# User Labeling

## Présentation
Le script en question a pour objectif de labéliser les utilisateurs en deux catégories distinctes : ceux qui sont considérés comme atypiques (avec un label à 1) et ceux qui ne le sont pas (avec un label à 0). 

Les utilisateurs jugés atypiques pourront être identifiés grâce à des comportements ou des actions qui sortent de l'ordinaire

En labélisant les utilisateurs de cette manière, le script peut permettre de mieux comprendre les comportements des utilisateurs.

Comment fonctionne-t-il ?
- Dès qu'un des attributs de l'utilisateur dépasse la valeur minimale qui lui est associée, celui-ci est alors considéré comme atypique.
- Une fois que le label d'un utilisateur a été identifié et stocké dans la collection `users_labeled`, nous pouvons récupérer les données standardisées présente dans ``users_scaled`` pour mettre à jour les attributs de l'utilisateur dont on a déterminé le label précédemment. On récupère les attributs de `users_scaled` et on ne standardise pas une nouvelle fois les données pour conserver les mêmes données. Si on avait standardisé sur la collection avec les utilisateurs labellisés, les attributs de l'utilisateur auraient changé de valeur du fait d'une moyenne et d'un écart-type différent.

## Le code
On importe les différentes bibliothèques nécessaires.

In [1]:
import pymongo
import math
import pandas as pd

Connexion à la base de données MongoDB

In [2]:
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client["Tweet"]

user_collection = db["users"]
user_scaled_collection = db["users_scaled"]
user_labeled_collection = db["users_labeled"]

On supprime toute la collection des utilisateurs labellisés pour supprimer par la même occasion les données qu'elle contient.

In [3]:
user_labeled_collection.drop()

On définit les valeurs minimales des champs sur lesquels on va déterminer si un utilisateur est atypique ou non

In [4]:
min_values_suspicious = {
    'hashtag_frequency': 5,
    'favourites_count': 60_000,
    'friends_count': 6_000,
    'followers_count': 9_000,
    'ratio_friends_followers': 10,
    'tweet_frequency': 70,
    'Ap': 0.02,
    'ratio_sensitive_tweets': 0,
    'ratio_punctuation_tweets': 0.5,
    'visibility': 0.5,
    'verified':0
}

Définition de la fonction qui détermine le score de suspicion d'un utilisateur

In [5]:
def get_suspicious_score(user):
    suspicious_fields = []
    for key, value in min_values_suspicious.items():
        if user[key] > value:
            suspicious_fields.append(key)
    return (len(suspicious_fields), suspicious_fields)

Définition de la fonction qui détermine le label d'un utilisateur enfocntion de on score de suspicion

In [6]:
def get_label(user):
    suspicious_score, suspicious_fields = get_suspicious_score(user)
    if suspicious_score >= 2:
        return (suspicious_score, suspicious_fields, 1)
    else:
        return (suspicious_score, suspicious_fields, 0)

Définition du nombre d'utilisateur à labéliser

In [7]:
nb_users_labeled = 20_000

On insérer dans mongodb les utilisateurs avec leur label calculé

In [8]:
users = user_collection.find({}).limit(nb_users_labeled)

users_with_label = []
for user in users:
    suspicious_score, suspicious_fields, label = get_label(user)
    
    user_labeled = {}
    user_labeled['_id'] = user['_id']
    user_labeled['user_id'] = user['user_id']
    user_labeled['label'] = label
    user_labeled['suspicious_score'] = suspicious_score
    user_labeled['suspicious_fields'] = suspicious_fields

    users_with_label.append(user_labeled)

user_labeled_collection.insert_many(users_with_label)


## Standardisation des utilisateurs labélisés

In [9]:
pipeline = [
    {
        '$merge': {
            'into': 'users_labeled',
            'on': '_id',
            'whenMatched': 'merge',
            'whenNotMatched': 'discard'
        }
    }
]

user_scaled_collection.aggregate(pipeline)


## Quelques statistiques

Affichage de l'occurence de chaque attribut atypique

In [10]:
pipeline = [
    {
        '$unwind': '$suspicious_fields'
    },
    {
        '$group': {
            '_id': '$suspicious_fields',
            'count': {'$sum': 1}
        }
    },
    {
        '$sort': {'count': -1}
    }
]

In [11]:
results = list(user_labeled_collection.aggregate(pipeline))
for result in results:
    print(result['_id'], ':', result['count'])

tweet_frequency : 1157
followers_count : 1050
ratio_friends_followers : 843
favourites_count : 700
hashtag_frequency : 404
friends_count : 403
verified : 377
ratio_sensitive_tweets : 311
Ap : 290
visibility : 107


Regarder la répartition des points de suspicion

In [12]:
res = user_labeled_collection.aggregate([
    {
        "$group": {
            "_id": "$suspicious_score",
            "count": { "$sum": 1 }
        }
    },
    { 
        "$sort" : { 
            "count" : -1 
        } 
    }
])

for doc in res:
    print(doc['_id'], ':', doc['count'])

0 : 16042
1 : 2695
2 : 941
3 : 239
4 : 69
5 : 12
6 : 2


Regarder la répartition des labels atypique ou non

In [13]:
res = user_labeled_collection.aggregate([
    {
        "$group": {
            "_id": "$label",
            "count": { "$sum": 1 }
        }
    },
    { 
        "$sort" : { 
            "count" : -1 
        } 
    }
])

for doc in res:
    print('classe', doc['_id'], ':', doc['count'])

classe 0 : 18737
classe 1 : 1263
